In [2]:
import pandas as pd

Pandas é uma biblioteca para trabalhar com leitura e manipulação de dados. Utilizamos para leitura do arquivo de dados disponibilizado.


In [3]:
dataframe = pd.read_excel("dataset.xlsx")

In [4]:
dataframe

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Respiratory Syncytial Virus,Influenza A,Influenza B,Parainfluenza 1,CoronavirusNL63,Rhinovirus/Enterovirus,Mycoplasma pneumoniae,Coronavirus HKU1,Parainfluenza 3,Chlamydophila pneumoniae,Adenovirus,Parainfluenza 4,Coronavirus229E,CoronavirusOC43,Inf A H1N1 2009,Bordetella pertussis,Metapneumovirus,Parainfluenza 2,Neutrophils,...,Urine - Esterase,Urine - Aspect,Urine - pH,Urine - Hemoglobin,Urine - Bile pigments,Urine - Ketone Bodies,Urine - Nitrite,Urine - Density,Urine - Urobilinogen,Urine - Protein,Urine - Sugar,Urine - Leukocytes,Urine - Crystals,Urine - Red blood cells,Urine - Hyaline cylinders,Urine - Granular cylinders,Urine - Yeasts,Urine - Color,Partial thromboplastin time (PTT),Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,"Prothrombin time (PT), Activity",Vitamin B12,Creatine phosphokinase (CPK),Ferritin,Arterial Lactic Acid,Lipase dosage,D-Dimer,Albumin,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.022340,-0.517413,0.010677,0.102004,0.318366,-0.950790,-0.094610,-0.223767,-0.292269,1.482158,0.166192,0.357547,-0.625073,-0.140648,not_detected,not_detected,not_detected,not_detected,not_detected,detected,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,-0.619086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,detected,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

Devido a presença de campos com o valor NaN, vamos substituir esses valores por 0 para realizar o treinamento das árvores.

In [5]:
dataframe = dataframe.fillna(0)

# Exploração de Dados
A função describe() do pandas foi utilizada para mostrar uma visão geral a respeito dos dados estatísticos presentes na tabela. 

A função info() apresenta dados sobre a tabela em si (tipos de campos presentes, quantidade de colunas, etc).



In [6]:
dataframe.describe()

,Patient age quantile,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Mycoplasma pneumoniae,Neutrophils,Urea,Proteina C reativa mg/dL,Creatinine,Potassium,Sodium,Alanine transaminase,Aspartate transaminase,Gamma-glutamyltransferase,Total Bilirubin,Direct Bilirubin,Indirect Bilirubin,Alkaline phosphatase,Ionized calcium,Magnesium,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Fio2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),Rods #,Segmented,Promyelocytes,Metamyelocytes,Myelocytes,Myeloblasts,Urine - Density,Urine - Sugar,Urine - Red blood cells,Partial thromboplastin time (PTT),Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,"Prothrombin time (PT), Activity",Vitamin B12,Creatine phosphokinase (CPK),Ferritin,Arterial Lactic Acid,Lipase dosage,D-Dimer,Albumin,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
count,5644.000000,5644.000000,5644.000000,5644.000000,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5644.0,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03
mean,9.318391,0.013997,0.008859,0.007264,-2.335732e-10,-1.710835e-09,-3.770503e-11,7.894130e-10,8.985679e-10,-8.390814e-10,1.082473e-10,6.629928e-10,-7.075675e-10,-3.683047e-10,7.686216e-10,-4.432198e-10,-3.428930e-10,1.088413e-09,2.605525e-10,0.0,5.370286e-10,-4.695390e-10,2.492080e-10,-5.017987e-10,1.339890e-09,7.366095e-10,1.084123e-10,-2.178146e-11,-4.009769e-11,-8.976603e-11,3.887661e-10,9.768656e-11,-5.197849e-11,1.689714e-10,-3.696248e-11,-4.663213e-10,3.636844e-10,-2.590674e-11,-1.905878e-10,0.0,-2.197948e-10,8.658957e-11,-1.064322e-10,4.488302e-11,-4.603809e-10,1.267285e-10,1.478499e-10,2.376160e-10,0.0,-3.168213e-11,0.0,9.372630e-11,0.0,-1.584106e-11,-1.115475e-10,3.102208e-11,0.0,-1.056071e-11,-1.194680e-10,2.970200e-11,-7.920532e-12,-5.280355e-12,0.0,-1.320089e-11,-6.600441e-13,4.026270e-11,-7.920532e-12,1.980133e-12,-3.574553e-11,2.904195e-11,-1.181479e-10,1.650111e-11,2.244151e-11,2.508169e-11
std,5.777903,0.117489,0.093713,0.084929,3.268915e-01,3.268915e-01,3.266203e-01,3.258055e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.263489e-01,3.266203e-01,1.919891e-01,0.0,3.015113e-01,2.652407e-01,2.994472e-01,2.741119e-01,2.564082e-01,2.560624e-01,1.996808e-01,2.001240e-01,1.646610e-01,1.795894e-01,1.795894e-01,1.795894e-01,1.597446e-01,9.413042e-02,8.419280e-02,1.552439e-01

In [7]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Columns: 111 entries, Patient ID to ctO2 (arterial blood gas analysis)
dtypes: float64(70), int64(4), object(37)
memory usage: 4.8+ MB


Acessamos a coluna que contém o diagnóstico final e atribuimos à variável diagnostic.

In [8]:
diagnostic = dataframe["SARS-Cov-2 exam result"]

# Pré processamento dos dados

Removemos todas as colunas que não são do tipo numérico visto que a função de treino da floresta randômica não sabe lidar com dados do tipo *string*.

In [9]:
cols_to_remove = []

for col in dataframe.columns:
    try:
        _ = dataframe[col].astype(float)
    except ValueError:
        cols_to_remove.append(col)
        pass

# keep only the columns in dataframe that do not contain string
dataframe = dataframe[[col for col in dataframe.columns if col not in cols_to_remove]]

Verificando se todos as colunas restantes possuem valores numéricos e não nulos.

In [10]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Data columns (total 74 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Patient age quantile                                   5644 non-null   int64  
 1   Patient addmited to regular ward (1=yes, 0=no)         5644 non-null   int64  
 2   Patient addmited to semi-intensive unit (1=yes, 0=no)  5644 non-null   int64  
 3   Patient addmited to intensive care unit (1=yes, 0=no)  5644 non-null   int64  
 4   Hematocrit                                             5644 non-null   float64
 5   Hemoglobin                                             5644 non-null   float64
 6   Platelets                                              5644 non-null   float64
 7   Mean platelet volume                                   5644 non-null   float64
 8   Red blood Cells                                 

Atribuimos à variável predictors os dados de laboratório, ou seja, os dados das colunas restantes com exceção da coluna de diagnóstico e das colunas removidas no pré processamento por conterem valores não numéricos.

In [11]:
predictors = dataframe

In [12]:
predictors

,Patient age quantile,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Mycoplasma pneumoniae,Neutrophils,Urea,Proteina C reativa mg/dL,Creatinine,Potassium,Sodium,Alanine transaminase,Aspartate transaminase,Gamma-glutamyltransferase,Total Bilirubin,Direct Bilirubin,Indirect Bilirubin,Alkaline phosphatase,Ionized calcium,Magnesium,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Fio2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),Rods #,Segmented,Promyelocytes,Metamyelocytes,Myelocytes,Myeloblasts,Urine - Density,Urine - Sugar,Urine - Red blood cells,Partial thromboplastin time (PTT),Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,"Prothrombin time (PT), Activity",Vitamin B12,Creatine phosphokinase (CPK),Ferritin,Arterial Lactic Acid,Lipase dosage,D-Dimer,Albumin,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,13,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17,0,0,0,0.236515,-0.022340,-0.517413,0.010677,0.102004,0.318366,-0.950790,-0.094610,-0.223767,-0.292269,1.482158,0.166192,0.357547,-0.625073,-0.140648,0.0,-0.619086,1.198059,-0.147895,2.089928,-0.305787,0.862512,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,..

# Divisão do conjunto de dados em Teste e Treino
Utilizamos a biblioteca train_test_split do sklearn para dividir os dados em 30% teste e 70% treino.

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
predictors_train, predictors_test, diagnostic_train, diagnostic_test = train_test_split(predictors, diagnostic, test_size=0.30, random_state=88)

A função train_test_split() com o parâmetro test_size=0.30 retorna 4 valores que são 4 dataframes da respectiva ordem: 

*  70% dos dados para construir as árvores (preditores),
*  30% dos dados para teste (predictors_test),
*  os diagnósticos referentes aos 70% dos dados preditores (diagnostic_train) 
* e os diagnósticos referentes aos 30% dos dados de teste (diagnostic_test).

# Declaração da *Random Forest* e treinamento dela com os dados de treino

In [15]:
from sklearn.ensemble import RandomForestClassifier 

In [16]:
rf_model = RandomForestClassifier(n_estimators=100)

In [17]:
rf_model.fit(predictors_train,diagnostic_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Verificando a relevância das colunas dos preditores

Para verificar as 10 colunas (testes/variáveis) mais relevantes para o diagnóstico final, criamos um dataframe com duas colunas (o nome da feature e a respectiva importância no modelo). Depois, ordenamos em ordem da maior para a menor.

In [18]:
columns_relevance = pd.DataFrame({'feature': list(predictors.columns),
                   'importance': rf_model.feature_importances_}).\
                    sort_values('importance', ascending = False)


In [19]:
columns_relevance.head(n=10)

,feature,importance
0,Patient age quantile,0.188499
11,Leukocytes,0.073010
1,"Patient addmited to regular ward (1=yes, 0=no)",0.053351
6,Platelets,0.051008
16,Monocytes,0.049496
8,Red blood Cells,0.031968
22,Proteina C reativa mg/dL,0.031605
14,Eosinophils,0.024292
15,Mean corpuscular volume (MCV),0.024136
4,Hematocrit,0.023360


# Verificando a precisão da predição do diagnóstico dos pacientes

Importamos a função classification_report() da biblioteca sklearn para visualizar as diversas métricas de acurácia obtidas. 

In [20]:
from sklearn.metrics import classification_report

Usamos o método predict() para testar no nosso modelo criado e treinado (rf_model) o conjunto de 30% dos dados originais que foram separados para teste anteriormente.

In [21]:
rf_predict = rf_model.predict(predictors_test)

O método classification_report() compara o rótulo objetivo (diagnóstico) do conjunto de 30% do original com as previsões do modelo. Por fim, imprimimos o resultado desse método.

In [22]:
print(classification_report(diagnostic_test, rf_predict))

              precision    recall  f1-score   support

    negative       0.91      1.00      0.95      1529
    positive       0.89      0.05      0.09       165

    accuracy                           0.91      1694
   macro avg       0.90      0.52      0.52      1694
weighted avg       0.91      0.91      0.87      1694



# Análise dos resultados obtidos

Pode-se observar que o nosso modelo consegue obter com 91% de precisão, os casos em que o diagnóstico é negativo para COVID-19. Além disso, obteve 85% de precisão para prever os casos postivos de COVID-19.



In [23]:
diagnostic.value_counts()

negative    5086
positive     558
Name: SARS-Cov-2 exam result, dtype: int64


Entretanto, a métrica *recall* para o diagnóstico positivo está baixa, o que indica que os dados estão desbalanceados (existem muito mais diagnósticos negativos do que positivos). Isso pode ser verificado ao realizar a contagem dos resultados dos diagnósticos (PCR).

# Referências

*   Removendo strings de um dataframe: https://stackoverflow.com/questions/42335385/delete-every-column-that-contains-a-string-in-dataframe
*   Modelo do RandomForest do Scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
*   Leitura de arquivos Xlsx com pandas: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
*   Verificando relevância das colunas: https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76